In [4]:
# Import pandas library
import numpy as np
import pandas as pd
schools = pd.read_csv('PyCitySchools/Resources/schools_complete.csv') 
students = pd.read_csv('PyCitySchools/Resources/students_complete.csv') 

#Calculate total schools, students, and budget of district
school_count = schools['School ID'].count()
student_count = schools['size'].sum()
total_budget = schools['budget'].sum()

# Average math and reading scores
Avg_Math = students['maths_score'].mean()
Avg_Reading = students['reading_score'].mean()

# Calculate pass rates/Pct
year = students.year.sort_values().unique()
Math_Pass = students.loc[students['maths_score'] >= 50]
Math_Pct = len(Math_Pass)/ student_count * 100
Reading_Pass = students.loc[students['reading_score'] >= 50]
Reading_Pct = len(Reading_Pass)/ student_count * 100
overall_Pass = (Math_Pct + Reading_Pct)/2 

#district summary dataframe
Dist_School_summary = pd.DataFrame({"Total Schools": school_count,
                                "Total Students": student_count,
                                "Total Budget": total_budget,
                                "Average Math Score": Avg_Math,
                                "Average Reading Score": Avg_Reading,
                                "% Passing Math": Math_Pct,
                                "% Passing Reading": Reading_Pct,
                                "% Overall Passing Rate": [overall_Pass]
                                })

Dist_School_summary = Dist_School_summary[["Total Schools", "Total Students", "Total Budget", 
                                     "Average Math Score", "Average Reading Score", 
                                     "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

Dist_School_summary




,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,70.338192,69.980138,86.078632,84.426857,85.252744


In [42]:
# Merge schools and students
outputt = pd.merge(schools, students, on="school_name", how="left")
outputt = outputt.drop(['School ID', 'Student ID'], axis=1)

# Setting index
School_Name = students.school_name.sort_values().unique()
S_Year = students.year.sort_values().unique()
school_index = schools.set_index('school_name')
school_type = school_index['type']
school_students = school_index['size']
school_budget = school_index['budget']
psb = school_budget/school_students

# Setting index and groupby for dataframe
index = outputt.set_index('school_name')
grouped = index.groupby(['school_name'])

#Average math and reading score
school_avg_math = grouped['maths_score'].mean()
school_avg_read = grouped['reading_score'].mean()

#calculating math Pass 
total_stu = grouped['size'].count()
grouped_math = Math_Pass.groupby('school_name')
school_pass_math = grouped_math['maths_score'].count()/total_stu*100

#calculating reading Pass 
grouped_reading = Reading_Pass.groupby('school_name')
school_pass_read = grouped_reading['reading_score'].count()/total_stu*100
                             
# Calculating overall pass rate
overall_pass = (school_pass_math + school_pass_read)/2

# Creating dataframe for summary
D_school_summary = pd.DataFrame({"School Type": school_type,
                              "Total Students": school_students,
                              "Total School Budget": school_budget,
                              "Per Student Budget": psb,
                              "Average Math Score": school_avg_math,
                              "Average Reading Score": school_avg_read,
                              "% Passing Math": school_pass_math,
                              "% Passing Reading": school_pass_read,
                              "Overall Passing Rate": overall_pass})

# Reorganize school summary columns 
D_school_summary = D_school_summary[["School Type", "Total Students", "Total School Budget", 
                                 "Per Student Budget", "Average Math Score", "Average Reading Score", 
                                 "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]
D_school_summary.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,89.509646
Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,89.962325
Figueroa High School,Government,2949,1884411,639.0,68.698542,69.077993,81.654798,82.807731,82.231265
Ford High School,Government,2739,1763916,644.0,69.091274,69.572472,82.438846,82.219788,82.329317
Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,89.850136


In [43]:
# Top performing schools by passing rate
Top_Performers = D_school_summary.loc[D_school_summary['Overall Passing Rate'] > 85]
Top_Performers.sort_values(['Overall Passing Rate'], ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Independent,1858,1081356,582.0,71.657158,71.359526,90.850377,89.074273,89.962325
Griffin High School,Independent,1468,917500,625.0,71.788147,71.245232,91.212534,88.487738,89.850136
Bailey High School,Government,4976,3124928,628.0,72.352894,71.008842,91.639871,87.379421,89.509646
Holden High School,Independent,427,248087,581.0,72.583138,71.660422,89.929742,88.524590,89.227166
Wright High School,Independent,1800,1049400,583.0,72.047222,70.969444,91.777778,86.666667,89.222222


In [7]:
#Top Performers
Top_performer = D_school_summary.sort_values("Overall Passing Rate", ascending = False)
Top_performer.head().style.format({'Total Students': '{:}',
                           "Total School Budget": "${:,.2f}", 
                           "Per Student Budget": "${:.2f}", 
                           "% Passing Math": "{:2f}", 
                           "% Passing Reading": "{:2f}", 
                           "% Overall Passing Rate": "{:2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,89.962325
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,89.850136
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,89.509646
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,89.227166
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,89.222222


In [8]:
#bottom performing schools summary by passing rate
Worst_Performers = D_school_summary.loc[D_school_summary['Overall Passing Rate'] < 85]
Worst_Performers.sort_values(['Overall Passing Rate'], ascending=True).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Hernandez High School,Government,4635,3022020,652.0,68.874865,69.186408,80.949299,81.877023,81.413161
Huang High School,Government,2917,1910635,655.0,68.935207,68.910525,81.693521,81.453548,81.573534
Johnson High School,Government,4761,3094650,650.0,68.843100,69.039277,82.062592,81.978576,82.020584
Wilson High School,Independent,2283,1319574,578.0,69.170828,68.876916,82.785808,81.296540,82.041174
Figueroa High School,Government,2949,1884411,639.0,68.698542,69.077993,81.654798,82.807731,82.231265


In [150]:
# Worst Preformers 
worst_perform = Top_performer.tail()
worst_perform = worst_perform.sort_values('Overall Passing Rate')
worst_perform.style.format({'Total Students': '{: }', 
                       "Total School Budget": "${:,.2f}", 
                       "Per Student Budget": "${:.2f}", 
                       "% Passing Math": "{:2f}", 
                       "% Passing Reading": "{:2f}", 
                       "% Overall Passing Rate": "{:2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,81.413161
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,81.573534
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,82.020584
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,82.041174
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,82.231265


In [179]:
#Calculations into dataframe
average_math_df = pd.read_csv('../PyCitySchools/PyCitySchools/Resources/students_complete.csv')

# Group the data by 'School Name' and 'Year' and calculate the mean score for each group
score_by_school_year = average_math_df.groupby(['school_name','year'])['maths_score'].mean()
Math_Score_df = pd.DataFrame(score_by_school_year)

# Print the result
Math_Score_df

maths_score
school_name           year             
Bailey High School    9       72.493827
                      10      71.897498
                      11      72.374900
                      12      72.675097
Cabrera High School   9       72.321970
                      10      72.437768
                      11      71.008299
                      12      70.604712
Figueroa High School  9       68.477804
                      10      68.331586
                      11      68.811001
                      12      69.325282
Ford High School      9       69.021609
                      10      69.387006
                      11      69.248862
                      12      68.617811
Griffin High School   9       72.789731
                      10      71.093596
                      11      71.692521
                      12      71.469178
Hernandez High School 9       68.586831
                      10      68.867156
                      11      69.154412
                      12      68.985075
Holden High School    9       70.543307
                      10      75.105263
                      11      71.640777
                      12      73.409639
Huang High School     9       69.081754
                      10      68.533246
                      11      69.431345
                      12      68.639316
Johnson High School   9       69.469286
                      10      67.990220
                      11      68.637730
                      12      69.287393
Pena High School      9       71.996364
                      10      72.396000
                      11      72.523438
                      12      71.187845
Rodriguez High School 9       71.940722
                      10      71.779808
                      11      72.364811
                      12      72.154626
Shelton High School   9       72.932075
                      10      72.506696
                      11      70.097087
                      12      72.331536
Thomas High School    9       69.234273
                      10      70.057007
                      11      69.657831
                      12      69.369822
Wilson High School    9       69.212361
                      10      69.455446
                      11      68.378965
                      12      69.787472
Wright High School    9       71.741176
                      10      72.179012
                      11      73.275862
                      12      70.848238

In [180]:
#Calculations into dataframe
average_read_df = pd.read_csv('../PyCitySchools/PyCitySchools/Resources/students_complete.csv')

# Group the data by 'School Name' and 'Year' and calculate the mean score for each group
score_by_school_year = average_read_df.groupby(['school_name','year'])['reading_score'].mean()
read_Score_df = pd.DataFrame(score_by_school_year)
read_Score_df


reading_score
school_name           year               
Bailey High School    9         70.901920
                      10        70.848265
                      11        70.317346
                      12        72.195525
Cabrera High School   9         71.172348
                      10        71.328326
                      11        71.201245
                      12        71.856021
Figueroa High School  9         70.261682
                      10        67.677588
                      11        69.152327
                      12        69.082126
Ford High School      9         69.615846
                      10        68.988701
                      11        70.735964
                      12        68.849722
Griffin High School   9         72.026895
                      10        70.746305
                      11        72.385042
                      12        69.434932
Hernandez High School 9         68.477569
                      10        70.621842
                      11        68.418199
                      12        69.244136
Holden High School    9         71.598425
                      10        71.096491
                      11        73.310680
                      12        70.481928
Huang High School     9         68.670616
                      10        69.516297
                      11        68.740638
                      12        68.671795
Johnson High School   9         68.719286
                      10        69.295029
                      11        69.969115
                      12        67.992521
Pena High School      9         70.949091
                      10        72.324000
                      11        71.703125
                      12        71.513812
Rodriguez High School 9         70.902921
                      10        70.137500
                      11        71.424453
                      12        71.414449
Shelton High School   9         70.715094
                      10        69.879464
                      11        71.150485
                      12        69.070081
Thomas High School    9         69.672451
                      10        69.741093
                      11        70.749398
                      12        68.730769
Wilson High School    9         68.683043
                      10        68.412541
                      11        68.796327
                      12        69.888143
Wright High School    9         71.823529
                      10        71.386831
                      11        70.475862
                      12        69.821138

In [46]:
#Scores by School Spending
D_school_summary = pd.DataFrame({"School Type": school_type,
                              "Total Students": school_students,
                              "Total School Budget": school_budget,
                              "Per Student Budget": psb,
                              "Average Math Score": school_avg_math,
                              "Average Reading Score": school_avg_read,
                              "% Passing Math": school_pass_math,
                              "% Passing Reading": school_pass_read,
                              "Overall Passing Rate": overall_pass})
# Creating bins
bins = [0,585,615,645,675]

# Creating names for bins
spending_range = ['<$585','$585-615','$615-645','$645-675']

# Change formatting to float so it can be binned 
D_school_summary['Per Student Budget'] = D_school_summary['Per Student Budget'].replace('$', '')
D_school_summary['Per Student Budget'] = D_school_summary['Per Student Budget'].astype(float)
                                                      
D_school_summary["Spending Range Per Student"] = pd.cut(D_school_summary["Per Student Budget"], 
                                                         bins, labels=spending_range)
spend_summary = D_school_summary.groupby("Spending Range Per Student")
spend_summary = spend_summary[["Average Math Score", "Average Reading Score", "% Passing Math", 
                               "% Passing Reading", "Overall Passing Rate"]]
spend_summary.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Range Per Student,,,,,
<$585,71.364587,70.716577,88.835926,86.390517,87.613222
$585-615,72.061215,70.935557,91.611445,86.651266,89.131356
$615-645,70.593378,70.268222,86.932827,85.153583,86.043205
$645-675,68.884391,69.045403,81.568470,81.769716,81.669093


In [9]:
#Scores by School Size

#creating bins
R_bins = [0, 1000, 2000, 5000]

# Creating names for bins
R_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"] 
D_school_summary["size"] = pd.cut(D_school_summary["Total Students"],R_bins, labels = R_labels)

Size_summary = D_school_summary.groupby("size")
Size_summary = Size_summary[["Average Math Score", "Average Reading Score", "% Passing Math", 
                               "% Passing Reading", "Overall Passing Rate"]]
Size_summary.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,89.182190
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,88.280354
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,83.776994
